# Test Vietnamese Stock Env

In [1]:
import os
from pathlib import Path
path = Path(os.getcwd())
os.chdir(path.parent.absolute())

# import warnings
# warnings.filterwarnings('ignore')
import pandas as pd
import pandas_ta as ta
import gym
import mt4_hst
from stable_baselines3.common.env_checker import check_env
from stock_env.feature.feature_extractor import *

path = "../stock_datasets/"

c:\Users\manbnm\Miniconda3\envs\tradingenv\lib\site-packages\pyfolio\pos.py:27: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; mutltipliers will not be applied' +


In [2]:
ticker = "FPT"
df = mt4_hst.read_hst(f"{path}/{ticker}1440.hst")

# Test VietnamStockEnv

In [ ]:
from stock_env.envs.vn_stock_env import VietnamStockEnv

env = VietnamStockEnv(
    df = df,
    feature_extractor = OneStockFeatures(),
    ticker=ticker)
check_env(env)

# play random
obs = env.reset(current_tick=0)
done = False
while not done:
    obs, _, done, _ = env.step(env.action_space.sample())
history = env.get_history()

In [ ]:
cols = 'time open close delta_shares quantity portfolio_value nav cash delta_vt'.split()
history[cols].head()

# Test Vietnam Stock Continuous Env

In [ ]:
from stock_env.envs.vn_stock_env import VietnamStockContinuousEnv

ticker = "FPT"

env = VietnamStockContinuousEnv(
    df = df,
    feature_extractor = OneStockFeatures(),
    ticker=ticker)
check_env(env)

# play random
obs = env.reset(current_tick=0)
done = False
while not done:
    obs, _, done, _ = env.step(env.action_space.sample())
history = env.get_history()

In [ ]:
cols = 'time open close actions delta_shares quantity portfolio_value nav cash delta_vt'.split()
history[cols].head()

# Test MultiStockEnv

In [ ]:
from stock_env.envs.multi_stock import MultiStockEnv

tickers = "FPT SSI VNM".split()
env = MultiStockEnv(
    tickers=tickers,
    feature_extractor=OneStockFeatures(),
    data_folder_path=path)
check_env(env)

obs = env.reset(current_tick=0)
done = False
while not done:
    obs, _, done, _ = env.step(env.action_space.sample())
histories, ticker_history = env.get_history()
histories.head()

In [ ]:
cols = 'time close delta_shares quantity portfolio_value nav cash delta_vt step_reward'.split()
ticker_history['FPT'][cols].head()

In [ ]:
ticker_history['SSI'][cols].head()

# Test Random Stock Continuous Env

In [ ]:
# Test Vietnam Stock Continuous Env
from stock_env.envs.random_stock import RandomStockEnv
tickers = "SSI HPG VNM".split()
data_folder_path = "../stock_datasets/"

env = RandomStockEnv(
    tickers = tickers,
    data_folder_path = data_folder_path,
    feature_extractor = OneStockFeatures())
check_env(env)

# play random
obs = env.reset(current_tick=0)
done = False
while not done:
    obs, _, done, _ = env.step(env.action_space.sample())
history = env.get_history()
history['percent_cash'] = history['cash'] / history['portfolio_value']

In [ ]:
cols = 'time ticker open close actions percent_cash delta_shares quantity portfolio_value nav cash'.split()
history[cols].head()